# Advent of Code Day 02
Coded in Db2 SQL. See [readme](https://github.com/ecrooks/aoc2021_SQL) for more details on how to use. 

Env setup - only needs to run once on each machine

In [ ]:
!pip install ipython-sql
!pip install ibm_db 
!pip install ibm_db_sa

Restart the Kernel if this is your first time installing the above. The next steps will fail unless you do this.

Import modules and load the SQL magic

In [ ]:
import ibm_db
import ibm_db_sa
import sqlalchemy
%load_ext sql
import getpass
import csv

In [ ]:
# get db2.ipynb at https://github.com/IBM/db2-jupyter/blob/master/db2.ipynb
# much easier to work with stored procedures with these tools
%run db2.ipynb

Connect to the database. Change the values of user, host, and password to match your environment. For connection to a local host, use 'localhost' for the host name. Also change the port number and database name in the connection string.

In [ ]:
user='db2inst1'
host='localhost'
dbname='aocdb'


password = getpass.getpass('Enter password for '+user+' on '+dbname)

#%sql db2+ibm_db://$user:$password@$host:50000/$dbname
%sql CONNECT TO $dbname USER $user USING $password HOST $host PORT 50000

## Day 2, Part 1 Solution

### Problem statement: 

>--- Day 2: Dive! ---
>Now, you need to figure out how to pilot this thing.  
>
>It seems like the submarine can take a series of commands like forward 1, down 2, or up 3:  
>
>forward X increases the horizontal position by X units.  
>down X increases the depth by X units.  
>up X decreases the depth by X units.  
>Note that since you're on a submarine, down and up affect your depth, and so they have the opposite result of what you might expect.  
>
>The submarine seems to already have a planned course (your puzzle input). You should probably figure out where it's >going. For example:  
>  
>forward 5  
>down 5  
>forward 8  
>up 3  
>down 8  
>forward 2  
>Your horizontal position and depth both start at 0. The steps above would then modify them as follows:  
>  
>forward 5 adds 5 to your horizontal position, a total of 5.  
>down 5 adds 5 to your depth, resulting in a value of 5.  
>forward 8 adds 8 to your horizontal position, a total of 13.  
>up 3 decreases your depth by 3, resulting in a value of 2.  
>down 8 adds 8 to your depth, resulting in a value of 10.  
>forward 2 adds 2 to your horizontal position, a total of 15.  
>After following these instructions, you would have a horizontal position of 15 and a depth of 10. (Multiplying these together produces 150.)  
>  
>Calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?  

### Create Objects and Load Data

Create table to hold and work with the Data. Order actually doesn't matter here, but I'm a DBA and incapable of creating a table without a primary key if I have a choice, and I don't know if order might matter for part2. We'll use an identity colum to track the order of the values. In SQL, we cannot depend on data coming back in the order we inserted it.

In [ ]:
%sql drop table aoc_day02;

In [ ]:
%%sql 
create table aoc_day02 (
	seq int generated by default as identity primary key
	, direction varchar(18) not null
	, units int not null);
create index ix_day02_direction on aoc_day02 (direction, units) allow reverse scans;

Import data into the table. Placing the file in the repo data directory will make it available in the right location if you use the container provided.

I wanted to use SQL for everything, but one thing Db2 can't handle in delimited files is a space as a delimiter. This means I have to parse the file in Python and write it out in some different format. Might as well be a csv.

In [ ]:
with open("data/day02_input_sample.del") as fin, open("data/day02_input_sample.csv", 'w') as fout:
    o=csv.writer(fout)
    for line in fin:
        o.writerow(line.split())

Sample data

In [ ]:
%%sql 
delete from aoc_day02;
call admin_cmd ('import from /database/day02_input_sample.csv of del 
    insert into aoc_day02 (direction, units)');

Quick verification

In [ ]:
%sql select count(*) from aoc_day02;

In [ ]:
%sql select * from aoc_day02 fetch first 5 rows only;

Full data

In [ ]:
with open("data/day02_input.del") as fin, open("data/day02_input.csv", 'w') as fout:
    o=csv.writer(fout)
    for line in fin:
        o.writerow(line.split())

In [ ]:
%%sql 
delete from aoc_day02;
call admin_cmd ('import from /database/day02_input.csv of del 
                    insert into aoc_day02 (direction, units)');

In [ ]:
%sql select count(*) from aoc_day02;

In [ ]:
# quick view of the data
%sql select * from aoc_day02 fetch first 10 rows only

### Final Solution to Part 1

In [ ]:
%%sql 
select direction, sum(units) as sum_units
    from aoc_day02 group by direction

Technically, I don't have to use a CTE here, but I believe using one will reduce the number of tablescans I'll have to do on the full data.

In [ ]:
%%sql 
with m as (select direction, sum(units) as sum_units
    from aoc_day02 group by direction)
select (select sum_units from m where direction='forward') as forward
        , (select sum_units from m where direction='down') - (select sum_units from m where direction='up') as depth
    from sysibm.sysdummy1;

In [ ]:
%%sql 
with m as (select direction, sum(units) as sum_units
    from aoc_day02 group by direction)
select (select sum_units from m where direction='forward') 
        * ((select sum_units from m where direction='down') - (select sum_units from m where direction='up')) as answer
    from sysibm.sysdummy1;

## Day 1, Part 2 Solution

### Problem statement: 
>Based on your calculations, the planned course doesn't seem to make any sense. You find the submarine manual and discover that the process is actually slightly more complicated.  
>  
>In addition to horizontal position and depth, you'll also need to track a third value, aim, which also starts at 0. The commands also mean something entirely different than you first thought:  
>  
>down X increases your aim by X units.  
>up X decreases your aim by X units.  
>forward X does two things:  
>It increases your horizontal position by X units.  
>It increases your depth by your aim multiplied by X.  
>Again note that since you're on a submarine, down and up do the opposite of what you might expect: "down" means aiming in the positive direction.  
>  
>Now, the above example does something different:  
>  
>forward 5 adds 5 to your horizontal position, a total of 5. Because your aim is 0, your depth does not change.  
>down 5 adds 5 to your aim, resulting in a value of 5.  
>forward 8 adds 8 to your horizontal position, a total of 13. Because your aim is 5, your depth increases by 8*5=40.  
>up 3 decreases your aim by 3, resulting in a value of 2.  
>down 8 adds 8 to your aim, resulting in a value of 10.  
>forward 2 adds 2 to your horizontal position, a total of 15. Because your aim is 10, your depth increases by 2*10=20 to a total of 60.  
>After following these new instructions, you would have a horizontal position of 15 and a depth of 60. (Multiplying these produces 900.)  
>  
>Using this new interpretation of the commands, calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?  

As I suspected, order now matters. I think I'll use a stored procedure here, as SQL PL works better for this problem. 

### Final Solution to Part 2

Note: this section will not work without the extensions in db2.ipynb

In [ ]:
%%sql -d
create or replace procedure db2inst1.sp_aoc_day02_part2 ( INOUT v_position INT
                                                        , INOUT v_depth INT
                                                        , INOUT v_aim INT
                                                        , OUT v_ans INT )
        not deterministic
        language sql
        reads sql data
BEGIN
        -- declare variables
        declare v_aim int;
        -- set isolation level
        set current isolation = UR;
        --set staring values
        set v_position = coalesce(v_position,0);
        set v_aim = coalesce(v_aim,0);
        set v_depth = coalesce(v_depth,0);
        -- query table and process row by row
        for for_routine as c_aoc_day02_data cursor for
                select direction, units from aoc_day02 order by seq
        do
                if direction = 'forward' THEN
                        set v_position = v_position + units;
                        set v_depth = v_depth + (v_aim * units);
                end if;
                if direction = 'down' THEN
                        set v_aim = v_aim + units;
                end if;
                if direction = 'up' THEN
                        set v_aim = v_aim - units;
                end if;
        end for;
        set v_ans = v_position * v_depth;
END @

In [ ]:
pos=0
dep=0
aim=0
ans=0
pos,dep,aim,ans=%sql -r call db2inst1.sp_aoc_day02_part2 (:pos,:dep,:aim,:ans)
display("The horizontal position is: "+pos)
display("The depth is: "+dep)
display("The answer is: "+ans)